In [5]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils import get_column_letter
import os
import sys

# --- Configuración ---
file_path = "Actualización Censo Resguardo Indigena de Guachucal - Eliminacion de sisben.xlsx"
sheet_index = 0
rows_to_skip = 6
sisben_col = 'SISBEN'
familia_col = 'FAMILIA'
integrantes_col = 'INTEGRANTES'
id_col = 'NUMERO DOCUMENTO'
fecha_nac_col = 'FECHA DE NACIMIENTO'
# *** NUEVO: Añadir nombre exacto columna teléfono ***
# >>> ¡Verifica que este sea el nombre EXACTO en tu Excel! <<<
telefono_col = 'TELEFONO'

# --- Cargar Datos ---
print(f"Intentando cargar archivo: {file_path}")
try:
    # *** IMPORTANTE: Leer IDs y Teléfono como TEXTO desde el inicio ***
    df = pd.read_excel(file_path,
                       sheet_name=sheet_index,
                       skiprows=rows_to_skip,
                       header=0,
                       dtype={
                           id_col: str,
                           telefono_col: str # <--- Lee Teléfono como texto
                       }
                      )
    print(f"Archivo Excel cargado. {len(df)} filas leídas inicialmente.")
except FileNotFoundError:
    print(f"Error Crítico: No se encontró el archivo en la ruta: {file_path}")
    sys.exit(1)
except Exception as e:
    print(f"Error Crítico al leer el archivo Excel: {e}")
    sys.exit(1)

# --- Limpieza Inicial y Preparación ---
df.columns = df.columns.str.strip()
print("Nombres de columnas limpiados.")

# *** Actualizar columnas requeridas ***
required_cols = [sisben_col, familia_col, integrantes_col, id_col, fecha_nac_col, telefono_col] # Añadido telefono_col
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"Error Crítico: Faltan columnas esenciales en el archivo: {missing_cols}")
    print(f"Columnas encontradas: {df.columns.tolist()}")
    sys.exit(1)
else:
    print("Columnas requeridas encontradas.")

# *** Conversión y limpieza de tipos de datos específicos ***
# Asegurar ID como texto y quitar '.0'
df[id_col] = df[id_col].astype(str).str.replace(r'\.0$', '', regex=True)
print(f"Columna '{id_col}' asegurada como texto.")

# *** NUEVO: Asegurar Teléfono como texto y quitar '.0' ***
df[telefono_col] = df[telefono_col].astype(str).str.replace(r'\.0$', '', regex=True).str.strip() # Añadir .strip() por si acaso
print(f"Columna '{telefono_col}' asegurada como texto.")

# Convertir FECHA DE NACIMIENTO a datetime
df[fecha_nac_col] = pd.to_datetime(df[fecha_nac_col], errors='coerce')
invalid_dates_count = df[fecha_nac_col].isnull().sum()
if invalid_dates_count > 0:
    print(f"Advertencia: Se encontraron {invalid_dates_count} valores inválidos en '{fecha_nac_col}'.")
else:
    print(f"Columna '{fecha_nac_col}' convertida a formato fecha/hora.")


# --- Filtrado por SISBEN y Separación ---
print(f"\n--- Filtrando registros por columna '{sisben_col}' y separando eliminados ---")
# ... (filtrado como estaba) ...
df[sisben_col] = df[sisben_col].astype(str).str.strip().str.upper()

count_before_filter = len(df)
count_si = len(df[df[sisben_col] == 'SI'])
print(f"Total de registros antes del filtro SISBEN: {count_before_filter}")
print(f"Registros con '{sisben_col}' = 'SI' (serán separados): {count_si}")

df_eliminados = df[df[sisben_col] == 'SI'].copy()
print(f"Se han separado {len(df_eliminados)} registros para el archivo de eliminados.")

df_filtrado = df[df[sisben_col] != 'SI'].copy()
count_after_filter = len(df_filtrado)
print(f"Total de registros conservados después del filtro SISBEN: {count_after_filter}")

if df_filtrado.empty:
  print("Advertencia: No quedaron registros para procesar en el archivo principal.")


# --- Limpieza Post-Filtro (Aplicada al DataFrame CONSERVADO) ---
if not df_filtrado.empty:
    print(f"\n--- Limpiando datos conservados ({len(df_filtrado)} filas) ---")
    # ... (resto de la limpieza como estaba) ...
    initial_rows_filtrado = len(df_filtrado)
    df_filtrado = df_filtrado.dropna(subset=[familia_col, integrantes_col])
    dropped_nan = initial_rows_filtrado - len(df_filtrado)
    if dropped_nan > 0:
        print(f"Se eliminaron {dropped_nan} filas adicionales de los conservados por tener '{familia_col}' o '{integrantes_col}' vacíos.")

    try:
        # ... (conversión familia/integrantes como estaba) ...
        df_filtrado[familia_col] = pd.to_numeric(df_filtrado[familia_col], errors='coerce')
        df_filtrado[integrantes_col] = pd.to_numeric(df_filtrado[integrantes_col], errors='coerce')

        rows_before_coerce_nan = len(df_filtrado)
        df_filtrado = df_filtrado.dropna(subset=[familia_col, integrantes_col])
        rows_after_coerce_nan = len(df_filtrado)
        if rows_before_coerce_nan > rows_after_coerce_nan:
            print(f"Se eliminaron {rows_before_coerce_nan - rows_after_coerce_nan} filas de los conservados porque '{familia_col}' o '{integrantes_col}' no pudieron convertirse a número.")

        df_filtrado[familia_col] = df_filtrado[familia_col].astype(int)
        df_filtrado[integrantes_col] = df_filtrado[integrantes_col].astype(int)
        print(f"Columnas '{familia_col}' e '{integrantes_col}' convertidas a entero en datos conservados.")

    except Exception as e:
        print(f"Error Crítico al convertir '{familia_col}' o '{integrantes_col}' a entero en los datos conservados: {e}")
        sys.exit(1)
else:
    print("\nSkipping limpieza post-filtro porque no hay datos conservados.")


# --- Ordenamiento y Reasignación de Código Familiar (Sobre datos CONSERVADOS) ---
if not df_filtrado.empty:
    print(f"\n--- Ordenando y Reasignando Códigos de Familia ({len(df_filtrado)} filas conservadas) ---")
    # ... (resto del ordenamiento y reasignación como estaba) ...
    print(f"Ordenando por la columna original: '{familia_col}'...")
    df_filtrado = df_filtrado.sort_values(by=familia_col).reset_index(drop=True)
    print("Datos conservados ordenados por código de familia original.")

    nuevo_codigo_col = 'Nuevo_Codigo_Familia_Secuencial'
    print(f"Reasignando códigos secuenciales en la columna '{nuevo_codigo_col}'...")
    nuevo_codigo_lista = []
    codigo_actual = 0
    ultimo_codigo_original_procesado = None

    for fam_orig in df_filtrado[familia_col]:
        if fam_orig != ultimo_codigo_original_procesado:
            codigo_actual += 1
            ultimo_codigo_original_procesado = fam_orig
        nuevo_codigo_lista.append(codigo_actual)

    df_filtrado[nuevo_codigo_col] = nuevo_codigo_lista
    print(f"Códigos de familia secuenciales reasignados en datos conservados. Rango: 1 a {codigo_actual}")
else:
     print("\nSkipping ordenamiento y reasignación porque no hay datos conservados.")

# --- Validación del Número de Integrantes (Sobre datos CONSERVADOS) ---
if not df_filtrado.empty:
    print("\n--- Validando número de integrantes en datos conservados ---")
    # ... (resto de la validación como estaba) ...
    integrantes_actuales_col = 'Integrantes_Actuales_PostFiltro'
    error_col = 'Error_Conteo_Integrantes'

    df_filtrado[integrantes_actuales_col] = df_filtrado.groupby(nuevo_codigo_col)[nuevo_codigo_col].transform('count')
    print(f"Columna '{integrantes_actuales_col}' calculada.")

    def crear_mensaje_error_integrantes(row):
        integrantes_declarados = row[integrantes_col]
        integrantes_reales = row[integrantes_actuales_col]
        if pd.isna(integrantes_declarados) or pd.isna(integrantes_reales):
             return "ERROR - Dato faltante para comparar integrantes"
        if int(integrantes_reales) != int(integrantes_declarados):
            return f'ERROR - Declarados Originalmente: {int(integrantes_declarados)}, Contados Ahora: {int(integrantes_reales)}'
        return ''

    df_filtrado[error_col] = df_filtrado.apply(crear_mensaje_error_integrantes, axis=1)
    num_errors = (df_filtrado[error_col] != '').sum()
    print(f"Validación completada. Se encontraron {num_errors} familias conservadas con discrepancia en el número de integrantes.")
else:
    print("\nSkipping validación de integrantes porque no hay datos conservados.")


# --- Función para aplicar formatos de columna en Excel ---
def apply_column_formats(worksheet, column_formats):
    print(f"Aplicando formatos específicos a la hoja '{worksheet.title}'...")
    header_row = worksheet[1]
    col_indices = {}
    for col_idx, cell in enumerate(header_row, 1):
        if cell.value in column_formats:
            col_indices[cell.value] = col_idx

    if not col_indices:
        print("Advertencia: Ninguna de las columnas especificadas para formato se encontró.")
        return

    for col_name, col_idx in col_indices.items():
        format_string = column_formats[col_name]
        print(f" - Aplicando formato '{format_string}' a la columna '{col_name}' (índice {col_idx})...")
        for row_idx in range(2, worksheet.max_row + 1):
            cell = worksheet.cell(row=row_idx, column=col_idx)
            if cell.value is not None:
                 # Aplicar formato directamente, Excel manejará tipos básicos
                 cell.number_format = format_string
                 # Forzar texto explícitamente para columnas designadas con '@'
                 if format_string == '@':
                     cell.value = str(cell.value) # Asegurar que el valor sea string
                     # cell.data_type = 's' # Podría ser necesario en casos difíciles, pero a menudo no

    print("Formatos específicos aplicados.")

# --- Guardar Resultados ---
nombre_base = os.path.splitext(os.path.basename(file_path))[0]

output_file_eliminados = f"ELIMINADOS_SISBEN_SI_{nombre_base}.xlsx"
sheet_name_eliminados = "Eliminados SISBEN=SI"

output_file_conservados = f"FILTRADO_Y_VALIDADO_{nombre_base}.xlsx"
sheet_name_conservados = "Censo Filtrado y Ordenado"

# *** Actualizar los formatos deseados para incluir Teléfono ***
excel_formats = {
    id_col: '@',  # Formato de Texto para ID
    fecha_nac_col: 'yyyy-mm-dd', # Formato de fecha
    telefono_col: '@' # Formato de Texto para Teléfono
}

# --- Guardar archivo de ELIMINADOS (aplicando formatos) ---
print(f"\n--- Guardando archivo de registros eliminados en: {output_file_eliminados} ---")
if not df_eliminados.empty:
    try:
        with pd.ExcelWriter(output_file_eliminados, engine="openpyxl",
                           datetime_format='YYYY-MM-DD') as writer:
            # *** Asegurar ID y Teléfono como texto ANTES de guardar ***
            df_eliminados[id_col] = df_eliminados[id_col].astype(str).str.replace(r'\.0$', '', regex=True)
            df_eliminados[telefono_col] = df_eliminados[telefono_col].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
            df_eliminados.to_excel(writer, sheet_name=sheet_name_eliminados, index=False)

        workbook = load_workbook(output_file_eliminados)
        worksheet = workbook[sheet_name_eliminados]
        apply_column_formats(worksheet, excel_formats)
        workbook.save(output_file_eliminados)
        print(f"Archivo de eliminados guardado exitosamente ({len(df_eliminados)} registros) con formatos aplicados.")
    except Exception as e:
        print(f"Error al guardar o formatear el archivo de eliminados '{output_file_eliminados}': {e}")
else:
    print("No se encontraron registros con SISBEN='SI', no se generará el archivo de eliminados.")


# --- Guardar archivo principal de CONSERVADOS (aplicando formatos y resaltado) ---
print(f"\n--- Guardando archivo principal procesado en: {output_file_conservados} ---")
if not df_filtrado.empty:
    try:
        with pd.ExcelWriter(output_file_conservados, engine="openpyxl",
                            datetime_format='YYYY-MM-DD') as writer:
             # *** Asegurar ID y Teléfono como texto ANTES de guardar ***
             df_filtrado[id_col] = df_filtrado[id_col].astype(str).str.replace(r'\.0$', '', regex=True)
             df_filtrado[telefono_col] = df_filtrado[telefono_col].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
             df_filtrado.to_excel(writer, sheet_name=sheet_name_conservados, index=False)

        workbook = load_workbook(output_file_conservados)
        worksheet = workbook[sheet_name_conservados]

        # 1. Aplicar formatos de número/fecha/texto definidos
        apply_column_formats(worksheet, excel_formats)

        # 2. Aplicar resaltado de errores
        print("Aplicando formato de color a filas con error de conteo...")
        # ... (lógica de resaltado como estaba) ...
        fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")
        error_col_idx = -1
        if error_col in df_filtrado.columns:
             for col_num, cell in enumerate(worksheet[1], 1):
                 if cell.value == error_col:
                     error_col_idx = col_num
                     break
        else:
             print(f"Advertencia: La columna de error '{error_col}' no está presente para aplicar resaltado.")

        if error_col_idx != -1:
            rows_colored = 0
            for row_idx in range(2, worksheet.max_row + 1):
                cell_value = worksheet.cell(row=row_idx, column=error_col_idx).value
                if cell_value and isinstance(cell_value, str) and cell_value.startswith("ERROR"):
                    for col_idx in range(1, worksheet.max_column + 1):
                        worksheet.cell(row=row_idx, column=col_idx).fill = fill
                    rows_colored += 1
            print(f"Formato de color rojo aplicado a {rows_colored} filas con error de conteo.")
        elif error_col in df_filtrado.columns:
             print(f"Advertencia: No se encontró la columna de error '{error_col}' en la cabecera del Excel. No se aplicará resaltado.")


        workbook.save(output_file_conservados)
        print(f"Archivo principal guardado exitosamente en {output_file_conservados} con formatos y resaltado aplicados.")

    except Exception as e:
        print(f"Error Crítico al guardar o formatear el archivo principal '{output_file_conservados}': {e}")
        sys.exit(1)
else:
     print("No hay datos conservados para guardar en el archivo principal.")

print("\n--- Script finalizado ---")

Intentando cargar archivo: Actualización Censo Resguardo Indigena de Guachucal - Eliminacion de sisben.xlsx
Archivo Excel cargado. 8096 filas leídas inicialmente.
Nombres de columnas limpiados.
Columnas requeridas encontradas.
Columna 'NUMERO DOCUMENTO' asegurada como texto.
Columna 'TELEFONO' asegurada como texto.
Advertencia: Se encontraron 8 valores inválidos en 'FECHA DE NACIMIENTO'.

--- Filtrando registros por columna 'SISBEN' y separando eliminados ---
Total de registros antes del filtro SISBEN: 8096
Registros con 'SISBEN' = 'SI' (serán separados): 455
Se han separado 455 registros para el archivo de eliminados.
Total de registros conservados después del filtro SISBEN: 7641

--- Limpiando datos conservados (7641 filas) ---
Columnas 'FAMILIA' e 'INTEGRANTES' convertidas a entero en datos conservados.

--- Ordenando y Reasignando Códigos de Familia (7641 filas conservadas) ---
Ordenando por la columna original: 'FAMILIA'...
Datos conservados ordenados por código de familia origina